In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [8]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [9]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [10]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

/home/m-isonuma/topicsum/ncrp.py:25: RuntimeWarning: invalid value encountered in true_divide
  p_child = np.array(s_child_prior * s_child_likelihood) / np.sum(s_child_prior * s_child_likelihood)


0 10000 0 0 

/home/m-isonuma/topicsum/ncrp.py:54: RuntimeWarning: invalid value encountered in true_divide
  p_child = np.array(s_child_prior * s_child_likelihood) / np.sum(s_child_prior * s_child_likelihood)


AssertionError: 

In [11]:
%debug

> /home/m-isonuma/topicsum/ncrp.py(236)get_perplexity()
    234             probs_paths.append(probs_path)
    235 
--> 236         assert nearly_equal(np.sum([sum(probs_path.values()) for probs_path in probs_paths]), doc.config.n_depth)
    237 
    238         # Depth Probability for Each Word

ipdb> probs_path
{<ncrp.Topic object at 0x7fa0d777e320>: nan, <ncrp.Topic object at 0x7fa0cfc83320>: nan, <ncrp.Topic object at 0x7fa0cfc83198>: nan, <ncrp.Topic object at 0x7fa0cfccadd8>: nan, <ncrp.Topic object at 0x7fa0cfc833c8>: nan, <ncrp.Topic object at 0x7fa0cfc83550>: nan, <ncrp.Topic object at 0x7fa0cfccac50>: nan, <ncrp.Topic object at 0x7fa0cfc83278>: nan, <ncrp.Topic object at 0x7fa0cfc7ff98>: nan, <ncrp.Topic object at 0x7fa0cfc7fd68>: nan, <ncrp.Topic object at 0x7fa0cfccacc0>: nan, <ncrp.Topic object at 0x7fa0cfc83080>: nan, <ncrp.Topic object at 0x7fa0cfc7ff28>: nan, <ncrp.Topic object at 0x7fa0cfccab38>: nan, <ncrp.Topic object at 0x7fa0cfc7fe10>: nan, <ncrp.Topic object at 0x

In [16]:
topic_root.get_s_child_prior()

[1985, 3306, 912, 213, 642, 1070, 820, 134, 253, 603, 736, 461, 123, 0.01]

In [14]:
topic_root.get_s_child_likelihood(train_docs[0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
topic = topic_root
doc = train_docs[0]
from scipy.special import gammaln

if len(topic.children) > 0:
    children_cnt_words = np.concatenate([np.array([child.cnt_words for child in topic.children]), np.zeros([1, topic.config.n_vocab])], 0) # (Children+1) x Vocabulary
else:
    children_cnt_words = np.zeros([1, topic.config.n_vocab]) # (Children+1) x Vocabulary

cnt_words_doc = doc.cnt_words[None, :] # 1 x Vocabulary

logits_likelihood = gammaln(np.sum(children_cnt_words, -1) + topic.config.n_vocab*topic.config.eta) \
                    - np.sum(gammaln(children_cnt_words + topic.config.eta), -1) \
                    - gammaln(np.sum(children_cnt_words + cnt_words_doc, -1) + topic.config.n_vocab*topic.config.eta) \
                    + np.sum(gammaln(children_cnt_words + cnt_words_doc + topic.config.eta), -1)
s_child_likelihood = np.exp(logits_likelihood)



In [20]:
logits_likelihood

array([-4428.18725725, -4420.07920043, -4599.51801201, -4243.19227424,
       -4267.47753077, -4417.49479797, -4449.94021246, -4101.56171356,
       -4265.20929007, -4242.97753814, -4067.78760569, -4243.15727614,
       -4045.85749699, -3988.72628608])

In [21]:
s_child_likelihood

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])